In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from time import sleep
from datetime import date, timedelta

In [2]:
load_dotenv()
cid = os.getenv('client_id')
secret = os.getenv('client_secret')

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [5]:
# !pip install selenium
# !pip install webdriver-manager

import webbrowser
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

# gecko=GeckoDriverManager().install()

years = range(1970, 1973)
driver = webdriver.Firefox(GeckoDriverManager().install())

for year in years:   
#     webbrowser.open('https://open.spotify.com/search/top hits of ' + str(year))
#     page = requests.get('https://open.spotify.com/search/top hits of '+ str(year))
#     html = BeautifulSoup(page.text, 'html.parser')
#     link = html.find("div", class_="_gB1lxCfXeR8_Wze5Cx9")
    driver.get('https://open.spotify.com/search/top hits of ' + str(year))
    driver.find_element(By.CLASS_NAME, ("_gB1lxCfXeR8_Wze5Cx9"))
#     element.click()
        
#     webbrowser.open(link)
    

[WDM] - Downloading: 19.0kB [00:00, 6.58MB/s]                                                                          


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\pgmin\\.wdm\\drivers\\geckodriver\\win64\\0.32\\geckodriver.exe'

In [6]:
urls = ['37i9dQZF1DWXQyLTHGuTIz', '37i9dQZF1DX43B4ApmA3Ee', '37i9dQZF1DXaQBa5hAMckp']
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX38yySwWsFRT"
# track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_id)["items"]]

# for track in sp.playlist_tracks(playlist_id)["items"]:
#     #URI
#     track_uri = track["track"]["uri"]
    
#     #Track name
#     track_name = track["track"]["name"]
    
#     #Main Artist
#     artist_uri = track["track"]["artists"][0]["uri"]
#     artist_info = sp.artist(artist_uri)
    
#     #Name, popularity, genre
#     artist_name = track["track"]["artists"][0]["name"]
#     artist_pop = artist_info["popularity"]
#     artist_genres = artist_info["genres"]
    
#     #Album
#     album = track["track"]["album"]["name"]
    
#     #Popularity of the track
#     track_pop = track["track"]["popularity"]
    
# sp.audio_features(track_uri)[0]

playlist_id = playlist_link.split("/")[-1].split("?")[0]
username = 'Spotify'
results = sp.user_playlist(username, playlist_id, 'tracks')

In [7]:
playlist_tracks_data = results['tracks']
playlist_tracks_id = []
playlist_tracks_titles = []
playlist_tracks_artists = []
playlist_tracks_primary_artists = []

for track in playlist_tracks_data['items']:
    playlist_tracks_id.append(track['track']['id'])
    playlist_tracks_titles.append(track['track']['name'])
    # adds a list of all artists involved in the song to the list of artists for the playlist
    artist_list = []
    for artist in track['track']['artists']:
        artist_list.append(artist['name'])
    playlist_tracks_artists.append(artist_list)
    playlist_tracks_primary_artists.append(artist_list[0])

In [8]:
features = sp.audio_features(playlist_tracks_id)
# print(features)

In [9]:
songs_df = pd.DataFrame(data=features, columns=features[0].keys())

songs_df['title'] = playlist_tracks_titles
songs_df['primary_artist'] = playlist_tracks_primary_artists
songs_df['all_artists'] = playlist_tracks_artists

songs_df = songs_df[['id', 'title', 'primary_artist', 'all_artists',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'speechiness', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

songs_df["duration_s"] = songs_df.duration_ms.apply(lambda x : round(x/1000))
songs_df.drop("duration_ms", axis=1, inplace=True)
songs_df

,id,title,primary_artist,all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_s
0,4cOdK2wGLETKBW3PvgPWqT,Never Gonna Give You Up,Rick Astley,[Rick Astley],0.721,0.939,8,-11.823,1,0.0376,0.11500,0.000038,0.1080,0.914,113.309,4,214
1,2tUBqZG2AbRi7Q0BIrVrEj,I Wanna Dance with Somebody (Who Loves Me),Whitney Houston,[Whitney Houston],0.709,0.824,1,-8.824,1,0.0453,0.20700,0.000307,0.0888,0.867,118.818,4,291
2,7G7tgVYORlDuVprcYHuFJh,Don't Dream It's Over,Crowded House,[Crowded House],0.439,0.422,8,-17.227,1,0.0409,0.01480,0.000048,0.0697,0.551,81.833,4,237
3,6wpGqhRvJGNNXwWlPmkMyO,I Still Haven't Found What I'm Looking For,U2,[U2],0.564,0.774,1,-9.424,1,0.0368,0.01350,0.001910,0.0861,0.657,100.894,4,277
4,1aZLIbKEdsyqxyD6iNcrbA,Pour Some Sugar On Me,Def Leppard,[Def Leppard],0.267,0.953,1,-4.899,0,0.0960,0.00211,0.000000,0.2830,0.568,84.989,4,267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,28sxgM4sBLWbHq9KbOLuE8,Lean on Me,Club Nouveau,[Club Nouveau],0.738,0.670,0,-8.930,1,0.1320,0.18500,0.000000,0.0989,0.687,177.277,4,349
96,6a4mvE1CdiRMPyAvG8ByLN,Under The Boardwalk,Bruce Willis,[Bruce Willis],0.729,0.296,2,-14.321,0,0.0286,0.74700,0.000000,0.2320,0.660,114.434,4,183
97,0ow5yxh0ouMJ0om8EyEt8P,China In Your Hand,T'Pau,[T'Pau],0.329,0.527,10,-8.453,1,0.0340,0.39500,0.000044,0.1650,0.456,139.688,4,306
98,1pTqpcvO02DUSzdN85loMZ,"Running In The Family - 7"" Version",Level 42,[Level 42],0.816,0.647,11,-12.208,0,0.0414,0.23200,0.000000,0.1320,0.935,124.833,4,237


In [ ]:
# Add columns for genre
# Add columns for explicit

In [10]:
#write to csv
with open('./data/main.csv', 'w', encoding='utf-8') as f:
        songs_df.to_csv(f, header= True, index=True)